# Getting Started with cuDF

While the world’s data doubles each year, CPU computing has hit a brick wall with the end of Moore’s law. For the same reasons, scientific computing and deep learning has turned to NVIDIA GPU acceleration, data analytics and machine learning where GPU acceleration is ideal. 

NVIDIA created RAPIDS – an open-source data analytics and machine learning acceleration platform that leverages GPUs to accelerate computations. RAPIDS is based on Python, has pandas-like and Scikit-Learn-like interfaces, is built on Apache Arrow in-memory data format, and can scale from 1 to multi-GPU to multi-nodes. RAPIDS integrates easily into the world’s most popular data science Python-based workflows. RAPIDS accelerates data science end-to-end – from data prep, to machine learning, to deep learning. And through Arrow, Spark users can easily move data into the RAPIDS platform for acceleration.

In this notebook, we will also show how to get started with GPU DataFrames using cuDF in RAPIDS.

Table of Contents

* Loading data into a GPU DataFrame (GDF)
  * Loading data into a Pandas DataFrame
  * Converting a Pandas DataFrame to a GDF
* Working with the GDF
  * Take a look at the columns and their data types
  * Slice the GDF
  * Modify data types
  * Manipulate data with a user-defined function (UDF)
  * Sort the data
  * Filter the data
  * One-hot encode categorical columns
  * Split the data into training and validation sets
  * Turn the GDFs into matrices

## Loading data into a GPU DataFrame (GDF)

### Loading data into a Pandas DataFrame

It's easy to load almost any sort of data (json, csv, etc) into a Pandas DataFrame. <br>
For example, let's import some census data from a compressed CSV file on disk:

In [1]:
import pandas

# read data from csv file into pandas dataframe
df = pandas.read_csv('../data/ipums_easy.csv.gz', compression='gzip')

[Read more on using a Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html)

### Converting a Pandas DataFrame to a GDF

Next, we use our `pandas.DataFrame` and to create a `cudf.dataframe.DataFrame` object using the `from_pandas` method.

In [2]:
import cudf

# convert the Panda dataframe into a GPU dataframe
gdf = cudf.DataFrame.from_pandas(df)

And that's it! For the most part, working with GPU DataFrames will be the same as working with Pandas DataFrames. See the [cuDF documentation](https://cudf.readthedocs.io/en/latest/index.html) for more information.

## Working with the GDF

### Take a look at the columns and their data types

In [3]:
# print the columns and their datatypes in this gdf
gdf.dtypes

RECTYPE          int64
YEAR             int64
DATANUM          int64
SERIAL           int64
NUMPREC          int64
SUBSAMP          int64
HHWT             int64
HHTYPE           int64
REPWT          float64
CLUSTER          int64
ADJUST         float64
CPI99          float64
REGION           int64
STATEICP         int64
STATEFIP         int64
COUNTY         float64
COUNTYFIPS     float64
METRO          float64
METAREA        float64
METAREAD       float64
MET2013        float64
MET2013ERR     float64
CITY           float64
CITYERR        float64
CITYPOP        float64
PUMA           float64
PUMARES2MIG    float64
STRATA           int64
PUMASUPR       float64
CONSPUMA       float64
                ...   
REPWTP51       float64
REPWTP52       float64
REPWTP53       float64
REPWTP54       float64
REPWTP55       float64
REPWTP56       float64
REPWTP57       float64
REPWTP58       float64
REPWTP59       float64
REPWTP60       float64
REPWTP61       float64
REPWTP62       float64
REPWTP63   

### Slice the GDF

Woah! This GDF has a lot of columns, let's make it more manageable...

In [4]:
# only select certain columns (and overwrite the gdf)
column_names = [
    'INCEARN', 'PERWT', 'ADJUST', 'STATEICP', 'ROOMS', 'BEDROOMS',
     'PHONE', 'VEHICLES', 'RACE', 'SEX', 'AGE', 'VETSTAT'
]
gdf = gdf.loc[:, column_names]

# show the first 5 records of each column
print(gdf.head(5))

  INCEARN PERWT   ADJUST STATEICP ROOMS BEDROOMS PHONE ... VETSTAT
0    4000   618 1.018516       21     7        4     2 ...       1
1   36700   684 1.018516       21     7        4     2 ...       1
2   54000   618 1.018516       49     5        4     2 ...       2
3     900   609 1.018516       49     5        4     2 ...       1
4    2000   621 1.018516       49     5        4     2 ...       1
[4 more columns]


### Modify data types

In [5]:
gdf.dtypes

INCEARN       int64
PERWT         int64
ADJUST      float64
STATEICP      int64
ROOMS         int64
BEDROOMS      int64
PHONE         int64
VEHICLES      int64
RACE          int64
SEX           int64
AGE           int64
VETSTAT       int64
dtype: object

Looks like `INCEARN` and `PERWT` are integers when they should be floats. Let's fix that...

In [6]:
import numpy as np

# convert the following two int64 columns to float64 data type
gdf['INCEARN'] = gdf['INCEARN'].astype(np.float64)
gdf['PERWT'] = gdf['PERWT'].astype(np.float64)

# take another look
gdf.dtypes

INCEARN     float64
PERWT       float64
ADJUST      float64
STATEICP      int64
ROOMS         int64
BEDROOMS      int64
PHONE         int64
VEHICLES      int64
RACE          int64
SEX           int64
AGE           int64
VETSTAT       int64
dtype: object

### Manipulate data with a user-defined function (UDF)

`INCEARN` is a column in our dataset that supposedly represents income earned; however, it does not truly represent the amount of income earned when adjusted for inflation. The `ADJUST` column represents the dollar inflation factor, which we can use to adjust `INCEARN` to the amount that the individual would have earned during the calender year. In our dataset, `ADJUST` is constant over all rows.

Below, we will define a simple function `adjust_incearn` that takes `INCEARN` and and multiplies it by a constant - in this case, the dollar inflation factor. We'll use the `applymap` method in our `cudf.dataframe.DataFrame` object to apply an element-wise function to transform the values in the Column.

In [7]:
# define a function to adjust the incearn column
# so it more accurately represents income earned
adjust = gdf['ADJUST'][0]   # take constant from first row
print('adjustment factor: {}'.format(adjust))
def adjust_incearn(incearn):
    return adjust * incearn;

# apply it to the 'population' column
gdf['INCEARN'] = gdf['INCEARN'].applymap(adjust_incearn)

# drop the ADJUST column
gdf.drop_column('ADJUST')

# compute the mean
print('mean adjusted income: {}'.format(gdf['INCEARN'].mean()))

adjustment factor: 1.018516
mean adjusted income: 18637.0999154208


### Sort the data

Next, let's sort out data to do some light exploration.

In [8]:
# sort the gdf by the INCEARN column
gdf = gdf.sort_values(by='INCEARN', ascending=True)
# reset the index so we can use loc slicing later
gdf = gdf.reset_index()
print(gdf.head(5))

        INCEARN PERWT STATEICP ROOMS BEDROOMS PHONE VEHICLES ... VETSTAT
0 -10184.141484 538.0       53     4        3     2        2 ...       1
1 -10184.141484 614.0       71     7        4     2        2 ...       1
2 -10184.141484 511.0       45     9        5     2        2 ...       1
3 -10184.141484 453.0       45     9        5     2        2 ...       1
4 -10184.141484 593.0       53     9        5     2        5 ...       1
[3 more columns]


Looks like we have some negative income values. Let's filter those out...

### Filter the data

We'll use the `query` method to filter our dataset. The `query` method takes as argument a boolean expression very similar to the `query` method for the `pandas.DataFrame` class. However, the `cudf.dataframe.DataFrame` implementation uses Numba to compile a GPU kernel. 

For more information on the syntax for arguments into `query`, see the Pandas documentation: 

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.query.html

In [9]:
# how many records do we have?
print("{} = Original # of records".format(len(gdf)))

# filter out
gdf = gdf.query('INCEARN >= 0')

# how many records do we have left?
print("{} = New # of records".format(len(gdf)))

# sanity check...
print(gdf.head(5))

10000 = Original # of records
9985 = New # of records
  INCEARN PERWT STATEICP ROOMS BEDROOMS PHONE VEHICLES ... VETSTAT
15     0.0 559.0       49     5        4     2        3 ...       1
16     0.0 589.0       43     8        4     2        3 ...       1
17     0.0 617.0       43     5        3     2        1 ...       1
18     0.0 574.0       43     6        4     2        1 ...       2
19     0.0 616.0       43     6        4     2        1 ...       1
[3 more columns]


### One-hot encode categorical columns

Next, let's prepare our categorical columns. Machine learning models won't take as input strings as so we need to convert each column and its string representations to a numerical representation. The most common way to convert a Column with `n` elements and `k` unique categories to a numerical representation is to create a matrix of shape `n` by `k` and impute a 1 in cell `(i, j)` if the `ith` element is of category `j` and 0 otherwise, where $j \in k$. This is known as one-hot encoding.

In [10]:
# define the categorical columns
cat_cols = set(['STATEICP', 'RACE', 'SEX', 'VETSTAT'])
# store the unique values for each category column
uniques = {}

# iterate through each categorical column and one-hot
# encode it using the unique values it has
for k in cat_cols:
    uniques[k] = gdf[k].unique_k(k=1000)
    cats = uniques[k][1:]  # drop first
    gdf = gdf.one_hot_encoding(k, prefix=k, cats=cats)
    del gdf[k]
    
# we should see many more columns since the categorical
# columns will get expanded due to one-hot encoding
gdf.dtypes

INCEARN        float64
PERWT          float64
ROOMS            int64
BEDROOMS         int64
PHONE            int64
VEHICLES         int64
AGE              int64
STATEICP_2     float64
STATEICP_3     float64
STATEICP_4     float64
STATEICP_5     float64
STATEICP_6     float64
STATEICP_11    float64
STATEICP_12    float64
STATEICP_13    float64
STATEICP_14    float64
STATEICP_21    float64
STATEICP_22    float64
STATEICP_23    float64
STATEICP_24    float64
STATEICP_25    float64
STATEICP_31    float64
STATEICP_32    float64
STATEICP_33    float64
STATEICP_34    float64
STATEICP_35    float64
STATEICP_36    float64
STATEICP_37    float64
STATEICP_40    float64
STATEICP_41    float64
                ...   
STATEICP_51    float64
STATEICP_52    float64
STATEICP_53    float64
STATEICP_54    float64
STATEICP_56    float64
STATEICP_61    float64
STATEICP_62    float64
STATEICP_63    float64
STATEICP_64    float64
STATEICP_65    float64
STATEICP_66    float64
STATEICP_67    float64
STATEICP_68

### Split the data into training and validation sets

Next, let's split out data into an 80% train dataset and a 20% validation dataset.

In [11]:
# enforce float64 data type on ALL columns
for k in gdf.columns:
    gdf[k] = gdf[k].astype(np.float64)

# set the fractions for training and validation
fractions = {
    "train": 0.8,
    "valid": 0.2
}

# validation splitpoint
splitpoint = int(len(gdf) * fractions["train"])
print('splitpoint: {} of {} is {}'.format(fractions["train"], len(gdf), splitpoint))

# break the gdf up into training and validation sets
gdfs = {
    "train": gdf.loc[:splitpoint],
    "valid": gdf.loc[splitpoint:]
}
print('gdfs["train"] has {} rows'.format(len(gdfs["train"])))
print('gdfs["valid"] has {} rows'.format(len(gdfs["valid"])))

splitpoint: 0.8 of 9985 is 7988
gdfs["train"] has 7974 rows
gdfs["valid"] has 2012 rows


### Turn the GDFs into matrices

Lastly, we want to convert our GPU DataFrame to a GPU Matrix for usage as input to other machine learning libraries such as cuML and XGBoost. We can use the `as_gpu_matrix` method to facillitate this conversion.

In [12]:
# produce gpu matrices (to input to ML libraries, etc.)
# this step should not be necessary in the near future
# (should be able to use gdf as input)
matrices = {
    "train": {
        "x": gdfs["train"].as_gpu_matrix(columns=gdf.columns[1:]),
        "y": gdfs["train"].as_gpu_matrix(columns=[gdf.columns[0]])
    },
    "valid": {
        "x": gdfs["valid"].as_gpu_matrix(columns=gdf.columns[1:]),
        "y": gdfs["valid"].as_gpu_matrix(columns=[gdf.columns[0]])
    }
}

# check the matrix shapes (sanity check)
print('matrices["train"]["x"] shape:', matrices["train"]["x"].shape)
print('matrices["train"]["y"] shape:', matrices["train"]["y"].shape)
print('matrices["valid"]["x"] shape:', matrices["valid"]["x"].shape)
print('matrices["valid"]["y"] shape:', matrices["valid"]["y"].shape)

matrices["train"]["x"] shape: (7974, 67)
matrices["train"]["y"] shape: (7974, 1)
matrices["valid"]["x"] shape: (2012, 67)
matrices["valid"]["y"] shape: (2012, 1)
